<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/NB15_ML_SL_GLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Lineares Generalizados (GLM)

## Introdução ao GLM
* O que é um Modelo Linear Generalizado (GLM)?

    * É uma extensão do modelo de regressão linear que permite lidar com variáveis de resposta que não seguem uma distribuição normal.

* O GLM combina:
    * Previsores lineares: Soma ponderada das variáveis independentes.
    * Função de ligação: Relaciona o previsor linear à variável de resposta.
    * Distribuição da variável de resposta: Como Binomial, Poisson, Gamma, etc.

* Por que usar GLM?

    * Modelar dados com distribuições não normais.
    * Lidar com variáveis de resposta contínuas, discretas ou categóricas.
    * Tornar os modelos lineares mais flexíveis para diferentes problemas.

* Exemplo prático:

    * Modelar a probabilidade de um cliente comprar um produto (binomial).
    * Prever o número de chamadas em um call center por hora (Poisson).
    * Prever o número de sinistros para seguradoras (Poisson).


## Explicação teórica:

* Um Modelo Linear Generalizado (GLM) estende os modelos lineares simples para cobrir variáveis-alvo com distribuições diferentes da normal. Ele é composto por:
    * Um componente linear: $𝜂 = 𝛽_0 + 𝛽_1.𝑋_1 + ⋯ + 𝛽_𝑘.𝑋_𝑘$
    * Uma função de ligação que transforma o componente linear para se adequar à distribuição da variável-alvo.
    * Uma distribuição da família exponencial para modelar a variável-alvo (Binomial, Poisson, Gamma, etc.).

* Aplicação prática em seguros:
    * Modelar o número de sinistros por cliente em uma seguradora (distribuição Poisson).
    * Prever o custo médio de sinistros para apólices de seguro saúde (distribuição Gamma).

* Aplicação prática em bancos:
    * Modelar a probabilidade de inadimplência de um cliente em um empréstimo (distribuição Binomial, regressão logística).

# Componentes de um GLM
* Função de ligação: Relaciona o componente linear com a média da distribuição da variável-alvo.

    * Exemplo: Para uma variável binária (compra ou não compra de um seguro), a função de ligação logit é usada.
* Distribuição da variável de resposta:
    * Binomial: Para eventos com sucesso/falha.
    * Poisson: Para contagens ou taxas.
    * Gamma: Para valores contínuos positivos e assimétricos.

* Previsores lineares: Variáveis explicativas como idade, renda ou histórico de crédito.

* Exemplo prático: Em um banco, a probabilidade de inadimplência pode ser modelada com a regressão logística, onde a função de ligação logit transforma a probabilidade em uma escala linear.

# Tipos Comuns de GLM e Aplicações
## Regressão Logística (Binomial):

* Seguro: Modelar a probabilidade de renovação de uma apólice.
* Banco: Estimar a chance de um cliente atrasar um pagamento.

## Regressão de Poisson:

* Seguro: Modelar o número de sinistros por cliente.
* Banco: Estimar o número de transações mensais de um cliente.

## Regressão Gamma:

* Seguro: Estimar o custo médio de sinistros para seguros de saúde.
* Banco: Prever o valor médio de gastos no cartão de crédito.

## Regressão Gaussiana (Linear):

* Seguro: Prever o valor total de prêmios arrecadados em um período.
* Banco: Estimar o valor total de financiamentos concedidos.

Exemplo

In [ ]:
!pip install faker

In [ ]:
from faker import Faker
import pandas as pd
import numpy as np

# Configurando a seed e inicializando o Faker
np.random.seed(42)
fake = Faker()

# Gerando um DataFrame com dados simulados
data = {
    "idade": np.random.randint(18, 70, 2500),
    "renda_anual": np.random.normal(50000, 15000, 2500).clip(20000, 150000),
    "anos_experiencia": np.random.randint(1, 40, 2500),
    "historico_credito": np.random.choice(["bom", "ruim"], 2500, p=[0.8, 0.2]),
    "possui_casa": np.random.choice([0, 1], 2500, p=[0.7, 0.3]),
    "possui_carro": np.random.choice([0, 1], 2500, p=[0.6, 0.4]),
    "regiao": np.random.choice(["norte", "sul", "leste", "oeste"], 2500),
    "tipo_veiculo": np.random.choice(["sedan", "suv", "hatch", "pickup"], 2500),
    "gastos_cartao": np.random.normal(2000, 500, 2500).clip(500, 5000),
    "n_dependentes": np.random.randint(0, 5, 2500),
    "premio_seguro": np.random.normal(3000, 1000, 2500).clip(500, 10000),
    "sinistros": np.random.poisson(2, 2500).clip(0, 10),
    "inadimplencia": np.random.choice([0, 1], 2500, p=[0.9, 0.1]),
    "cidade": [fake.city() for _ in range(2500)],
    "estado": [fake.state_abbr() for _ in range(2500)],
}

# Criando o DataFrame
df_glm = pd.DataFrame(data)

# Exibindo o DataFrame para o usuário
df_glm.head()


# Ajuste dos modelos de GLM
* Ajustar Modelos Lineares Generalizados (GLMs) em Python é um processo simples e eficaz com bibliotecas como statsmodels e scikit-learn. Aqui está um guia passo a passo, incluindo exemplos baseados no dataset gerado.

### Importar as Bibliotecas
As principais bibliotecas utilizadas são:

* statsmodels para ajustar GLMs.
* pandas e numpy para manipulação de dados.
* scikit-learn para dividir os dados em treino e teste.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Configurar pandas para exibir números sem notação científica
pd.options.display.float_format = '{:.6f}'.format


## Escolha do GLM e da Distribuição
Os GLMs são ajustados com base:

* Na natureza da variável-alvo (distribuição).
* Na função de ligação que relaciona a predição do modelo com a variável-alvo.

* Exemplo:

    * Regressão Logística: Variável binária, como inadimplência (0 ou 1).
    * Regressão de Poisson: Variável de contagem, como número de sinistros.
    * Regressão Gamma: Variável contínua positiva, como prêmio de seguro.

## Ajuste do Modelo
### Modelando o número de sinistros

In [ ]:
# Definindo variáveis preditoras e alvo para Poisson
X_poisson = df_glm[["idade", "renda_anual", "anos_experiencia", "possui_casa", "possui_carro"]]
y_poisson = df_glm["sinistros"]

# Adicionando um intercepto
X_poisson = sm.add_constant(X_poisson)

# Dividindo os dados em treino e teste
X_train_poisson, X_test_poisson, y_train_poisson, y_test_poisson = train_test_split(
    X_poisson, y_poisson, test_size=0.3, random_state=42
)

# Ajustando o modelo de regressão de Poisson
model_poisson = sm.GLM(y_train_poisson, X_train_poisson, family=sm.families.Poisson())
result_poisson = model_poisson.fit()

# Exibindo o resumo do modelo
print(result_poisson.summary())

# Fazendo previsões no conjunto de teste
y_pred_poisson = result_poisson.predict(X_test_poisson)

# Avaliando o modelo com MAE
mae_poisson = np.mean(np.abs(y_test_poisson - y_pred_poisson))
print(f"Média dos erros absolutos (MAE) para Poisson: {mae_poisson}")


## Ajustando o Modelo de Regressão Gamma para modelar o Prêmio de Seguro

In [ ]:
# Definindo variáveis preditoras e alvo para Gamma
X_gamma = df_glm[["idade", "renda_anual", "anos_experiencia", "possui_casa", "possui_carro"]]
y_gamma = df_glm["premio_seguro"]

# Adicionando um intercepto
X_gamma = sm.add_constant(X_gamma)

# Dividindo os dados em treino e teste
X_train_gamma, X_test_gamma, y_train_gamma, y_test_gamma = train_test_split(
    X_gamma, y_gamma, test_size=0.3, random_state=42
)

# Ajustando o modelo de regressão Gamma
model_gamma = sm.GLM(y_train_gamma, X_train_gamma, family=sm.families.Gamma(link=sm.families.links.log()))
result_gamma = model_gamma.fit()

# Exibindo o resumo do modelo
print(result_gamma.summary())

# Fazendo previsões no conjunto de teste
y_pred_gamma = result_gamma.predict(X_test_gamma)

# Avaliando o modelo com MAE
mae_gamma = np.mean(np.abs(y_test_gamma - y_pred_gamma))
print(f"Média dos erros absolutos (MAE) para Gamma: {mae_gamma}")


## Visualização dos Resultados
* Regressão de Poisson: Valores Reais vs Preditos

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test_poisson, y_pred_poisson, alpha=0.5, label="Predito vs Real")
plt.xlabel("Valores Reais (Sinistros)")
plt.ylabel("Valores Preditos (Sinistros)")
plt.title("Regressão de Poisson: Valores Reais vs Preditos")
plt.legend()
plt.show()


* Regressão Gamma: Valores Reais vs Preditos

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test_gamma, y_pred_gamma, alpha=0.5, label="Predito vs Real")
plt.xlabel("Valores Reais (Prêmio de Seguro)")
plt.ylabel("Valores Preditos (Prêmio de Seguro)")
plt.title("Regressão Gamma: Valores Reais vs Preditos")
plt.legend()
plt.show()


## Avaliação do Modelo
Avalie o desempenho do modelo com métricas apropriadas:

* Log-Likelihood: Qualidade do ajuste.
* AIC (Akaike Information Criterion): Penaliza modelos complexos.
* Previsões e Taxas de Acerto: Compare as previsões com os valores reais.

## Interpretação dos Resultados
* Os coeficientes estimados indicam a relação entre as variáveis preditoras e a variável-alvo:

* Coeficientes positivos indicam aumento na variável-alvo à medida que a preditora aumenta.
* Coeficientes negativos indicam redução.

# Resumo
Ajustar GLMs em Python é poderoso para resolver problemas como:

* Regressão Logística: Probabilidade de inadimplência.
* Regressão de Poisson: Número de sinistros.
* Regressão Gamma: Custo de seguro.

# Como interpretar os coeficientes dos Modelos Lineares Generalizados?

A interpretação dos coeficientes em um **Modelo Linear Generalizado (GLM)** depende da **distribuição da variável-alvo** e da **função de ligação** usada no modelo. Aqui está um guia detalhado para interpretar coeficientes em diferentes contextos:

---

### **1. Noções Básicas**
- **Coeficiente $\beta$:** Representa a mudança esperada na variável-alvo quando a variável preditora aumenta em uma unidade, mantendo as outras constantes.
- **Intercepto $\beta_0$:** Valor médio esperado da variável-alvo quando todas as preditoras são zero (após a transformação da função de ligação).

**Nota:** Para variáveis categóricas, os coeficientes indicam a diferença no valor médio da variável-alvo em relação à categoria de referência.

---

### **2. Distribuições e Funções de Ligação**
A interpretação exata do coeficiente varia com a função de ligação.

#### **2.1. Regressão Logística (Binomial)**
- **Função de Ligação:** Logit ($logit_p = ln\left(\frac{p}{1-p}\right)$).
- **Interpretação do Coeficiente $\beta$:**
  - O coeficiente indica a mudança no logaritmo das odds (razão de chances) para cada unidade de aumento na variável preditora.
  - Para interpretá-lo como um efeito multiplicativo:
    $
    \text{Odds ratio} = e^\beta
    $
    - Se $e^\beta > 1$: Aumenta a chance do evento.
    - Se $e^\beta < 1$: Reduz a chance do evento.

**Exemplo:**
- Um coeficiente de $0.5$ para a variável "idade" significa que, para cada ano adicional de idade, a chance do cliente se tornar inadimplente aumenta em $e^{0.5} ≈ 1.65$ vezes (65% mais provável).

---

#### **2.2. Regressão de Poisson**
- **Função de Ligação:** Logarítmica $\ln(\mu)$.
- **Interpretação do Coeficiente $\beta$:**
  - O coeficiente indica a mudança no logaritmo da taxa esperada (ou contagem) para cada unidade de aumento na variável preditora.
  - Para interpretá-lo diretamente:
    $
    \text{Taxa esperada} = e^\beta
    $
    - Se $e^\beta > 1$: Aumenta a taxa esperada.
    - Se $e^\beta < 1$: Reduz a taxa esperada.

**Exemplo:**
- Um coeficiente de $-0.2$ para "anos de experiência" indica que, para cada ano adicional, o número esperado de sinistros diminui em cerca de $e^{-0.2} ≈ 0.82$ vezes (18% menor).

---

#### **2.3. Regressão Gamma**
- **Função de Ligação:** Logarítmica $\ln(\mu)$.
- **Interpretação do Coeficiente $\beta$:**
  - Similar à regressão de Poisson, mas a variável-alvo é contínua e positiva.
  - Cada unidade adicional na variável preditora multiplica o valor médio da variável-alvo por $e^\beta$.

**Exemplo:**
- Um coeficiente de (0.1) para "renda anual" indica que, para cada incremento de unidade, o custo médio do prêmio de seguro aumenta em $e^{0.1} \approx 1.11 $ vezes (11% maior).

---

#### **2.4. Regressão Linear (Distribuição Normal)**
- **Função de Ligação:** Identidade $\mu = X\beta$.
- **Interpretação do Coeficiente $\beta$:**
  - Indica a mudança absoluta na variável-alvo para cada unidade de aumento na variável preditora.

**Exemplo:**
- Um coeficiente de $200$ para "anos de experiência" significa que, para cada ano adicional, o valor do prêmio de seguro aumenta em $200$.

---

### **3. Interpretação de Variáveis Categóricas**
Para variáveis categóricas:
- Os coeficientes representam a diferença na média da variável-alvo em relação à **categoria de referência**.
- Se uma categoria tem coeficiente zero, ela é a referência.

**Exemplo:**
Na variável "região" com categorias "norte", "sul", "leste" e "oeste":
- Se "norte" é a categoria de referência, um coeficiente de $0.3$ para "sul" significa que a variável-alvo é $e^{0.3} \approx 1.35$ vezes maior para clientes na região sul em relação à região norte.

---

### **4. Interpretação do Intercepto**
- Para variáveis contínuas:
  - O intercepto representa o valor médio da variável-alvo quando todas as preditoras são zero (após a transformação).
- Para variáveis categóricas:
  - Ele representa o valor médio da categoria de referência.

---

### **5. Resumo da Interpretação**
| Distribuição        | Função de Ligação   | Interpretação do Coeficiente              |
|---------------------|---------------------|-------------------------------------------|
| Binomial (Logística)| Logit $\ln(p/(1-p))$ | Multiplicativo nas odds (razão de chances) |
| Poisson             | Log $\ln(\mu)$     | Multiplicativo na taxa esperada           |
| Gamma               | Log $\ln(\mu)$     | Multiplicativo na média esperada          |
| Normal (Linear)     | Identidade $\mu$    | Mudança absoluta na média esperada        |

Interpretar coeficientes no GLM ajuda a entender como cada variável preditora afeta a variável-alvo, fornecendo insights valiosos para negócios e seguros.